In [34]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import urllib.parse as urlparse
import requests
from matplotlib import rc
rc('font', family = "Arial Unicode MS")
# from urllib3 import request, urlopen
import urllib.request

### 데이터 로드

In [35]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

### train + test 붙이기

In [36]:
train = train.append(test)
train

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45,2021-04-05,월,2973,125,174,704,331.0,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN
46,2021-04-06,화,2973,76,170,636,364.0,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN
47,2021-04-07,수,2973,96,214,1,334.0,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN
48,2021-04-08,목,2973,105,238,509,324.0,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN


### 공휴일 데이터 받기

In [37]:
# 토, 일을 제외한 국경일 및 공휴일 추가
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
operation = "getRestDeInfo"
mykey = 'ac%2FSOW4KzOFKdu0z01iEVWGZd4TBl1MyiR04%2FfYmADthCjJBEyL73pewbapUk94Gm1%2FqtzMCban3C%2BpqU8c7ew%3D%3D'

date = []
datename = []
for year in range(2016, 2022):
    year = str(year)
    
    for month in range(1, 13):
        if month < 10:
            month = "0" + str(month)
        else:
            month = str(month)
            
        params = {'solYear' : year, 'solMonth' : month}
        rq_query = url +'/' + operation + '?' + urlparse.urlencode(params) + "&serviceKey=" + mykey    
        response = requests.get(rq_query) 
        dom = BeautifulSoup(response.content, "html.parser")
        
        items = dom.find_all("item")
        for item in items:
            date.append(item.locdate.string)
            datename.append(item.datename.string)

holiday_df= pd.DataFrame({"date": date, "datename": datename})

In [38]:
# 휴일 전날
holiday_before_df = holiday_df.copy()
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : x - relativedelta(days=1))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strftime(x, '%Y%m%d'))
holiday_before_df['datename'] = holiday_df['datename'].apply(lambda x : 1)

# 휴일 다음날
holiday_after_df = holiday_df.copy()
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : x + relativedelta(days=1))
holiday_after_df['date'] = holiday_after_df['date'].apply(lambda x : datetime.strftime(x, '%Y%m%d'))
holiday_after_df['datename'] = holiday_df['datename'].apply(lambda x : 1)

holiday_before_df.columns = ['일자', '휴일전날']
holiday_after_df.columns = ['일자', '휴일다음날']

In [39]:
# 휴일전날 merge
train_df = pd.merge(train, holiday_before_df, left_on='일자', right_on='일자', how='left')
before_friday = train_df[train_df['요일']=='금'].index
train_df['휴일전날'][before_friday] = 1
# 휴일 다음날 merge
train_df = pd.merge(train_df, holiday_after_df, left_on='일자', right_on='일자', how='left')
after_sunday = train_df[train_df['요일']=='월'].index
train_df['휴일다음날'][after_sunday] = 1

train_df['휴일전날'] = train_df['휴일전날'].fillna(0)
train_df['휴일다음날'] = train_df['휴일다음날'].fillna(0)

In [40]:
train_df

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2021-04-05,월,2973,125,174,704,331.0,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN,0.0,1.0
1251,2021-04-06,화,2973,76,170,636,364.0,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN,0.0,0.0
1252,2021-04-07,수,2973,96,214,1,334.0,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN,0.0,0.0
1253,2021-04-08,목,2973,105,238,509,324.0,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN,0.0,0.0


### 강수량

In [41]:
rain_df = pd.DataFrame()
for i in range(2016, 2022):
    tmp = pd.read_csv(f'./weather_data/{i}.csv', encoding='euc-kr')
    rain_df = rain_df.append(tmp)

In [42]:
rain_df = rain_df[['일시', '강수량(mm)']].rename(columns={'강수량(mm)':'강수량'}).reset_index(drop=True)
rain_df['일시'] = rain_df['일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M'))
rain_df['일자'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%Y-%m-%d'))
rain_df['시간'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%H'))
rain_df['시간'] = rain_df['시간'].astype('int')
rain_df = rain_df[['일자', '시간', '강수량']]

In [43]:
# 점심, 저녁 5미리 이상
lunch_rain_df = rain_df[(rain_df['시간'] == 12) & (rain_df['강수량'] >= 5)].reset_index(drop=True)
dinner_rain_df = rain_df[(rain_df['시간'] == 18) & (rain_df['강수량'] >= 5)].reset_index(drop=True)

In [44]:
# 점심비, 저녁비 merge
train_df = pd.merge(train_df, lunch_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'점심비'})

train_df = pd.merge(train_df, dinner_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'저녁비'})

In [45]:
train_df

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0,0.0,0.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0,0.0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0,0.0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0,0.0,0.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2021-04-05,월,2973,125,174,704,331.0,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN,0.0,1.0,0.0,0.0
1251,2021-04-06,화,2973,76,170,636,364.0,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN,0.0,0.0,0.0,0.0
1252,2021-04-07,수,2973,96,214,1,334.0,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN,0.0,0.0,0.0,0.0
1253,2021-04-08,목,2973,105,238,509,324.0,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN,0.0,0.0,0.0,0.0


### 원핫 인코딩(요일)

In [46]:
onehot_tmp = pd.get_dummies(train_df['요일'])
train_df = pd.concat([train_df.drop(columns='요일'), onehot_tmp], axis=1)

In [47]:
train_df

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비,금,목,수,월,화
0,2016-02-01,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0,1.0,0.0,0.0,0,0,0,1,0
1,2016-02-02,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0,0.0,0.0,0.0,0,0,0,0,1
2,2016-02-03,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0,0.0,0.0,0.0,0,0,1,0,0
3,2016-02-04,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0,0.0,0.0,0.0,0,1,0,0,0
4,2016-02-05,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1.0,0.0,0.0,0.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2021-04-05,2973,125,174,704,331.0,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN,0.0,1.0,0.0,0.0,0,0,0,1,0
1251,2021-04-06,2973,76,170,636,364.0,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN,0.0,0.0,0.0,0.0,0,0,0,0,1
1252,2021-04-07,2973,96,214,1,334.0,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN,0.0,0.0,0.0,0.0,0,0,1,0,0
1253,2021-04-08,2973,105,238,509,324.0,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN,0.0,0.0,0.0,0.0,0,1,0,0,0


### 밥먹을 인원만 추리기 -> X

In [58]:
# train_df['본사정원수'] = train_df['본사정원수'] - train_df['본사휴가자수'] - train_df['본사출장자수'] - train_df['현본사소속재택근무자수']

In [59]:
# train_df = train_df.drop(columns=['본사휴가자수', '본사출장자수', '현본사소속재택근무자수'])

### 코로나 데이터 추가

In [68]:
pd.read_csv('../yk/data/train_join_v3.csv', index_col = 0)[]

SyntaxError: invalid syntax (<ipython-input-68-0954850a4bc1>, line 1)

### 메뉴 데이터 추가 및 일자 삭제

In [50]:
lunch_train = pd.read_csv('../yk/data/lunch_train.csv')
lunch_test = pd.read_csv('../yk/data/lunch_test.csv')
dinner_train = pd.read_csv('../yk/data/dinner_train.csv')
dinner_test = pd.read_csv('../yk/data/dinner_test.csv')

In [51]:
# 트레인_테스트
lunch_menu = pd.concat([lunch_train, lunch_test], axis=0).reset_index(drop=True)
dinner_menu = pd.concat([dinner_train, dinner_test], axis=0).reset_index(drop=True)

# 임베딩만 남김
lunch_menu = lunch_menu.drop(columns=['중식메뉴'])
dinner_menu = dinner_menu.drop(columns=['석식메뉴'])

In [52]:
lunch_menu

,샐러드류,양념 및 장류,나물류,소스류,두류,찜류,곡류,죽류,국탕류,볶음류,...,해조류,과일류,전류,조림류,튀김류,장아찌류,구이류,스프류,덮밥국밥류,떡류
0,0,0,0,0,0,1,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,1,0,0,0,0,2,0,1,0,...,0,0,0,1,1,0,0,0,1,0
3,0,0,1,0,0,0,2,0,1,1,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,3,0,1,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,1,0,0,1,0,0,3,0,1,1,...,1,1,0,0,1,0,1,0,0,0
1251,1,0,0,1,1,1,3,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1252,1,2,0,1,0,0,3,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1253,1,1,0,2,0,1,4,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [53]:
# 일자, 메뉴 드랍
train_df = train_df.drop(columns='일자')
train_df = train_df.drop(columns='조식메뉴')
train_df = train_df.drop(columns='중식메뉴')
train_df = train_df.drop(columns='석식메뉴')

In [54]:
train_df.head()

,본사정원수,본사시간외근무명령서승인건수,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비,금,목,수,월,화
0,2401.0,238,1039.0,331.0,0.0,1.0,0.0,0.0,0,0,0,1,0
1,2378.0,319,867.0,560.0,0.0,0.0,0.0,0.0,0,0,0,0,1
2,2365.0,111,1017.0,573.0,0.0,0.0,0.0,0.0,0,0,1,0,0
3,2277.0,355,978.0,525.0,0.0,0.0,0.0,0.0,0,1,0,0,0
4,2142.0,34,925.0,330.0,1.0,0.0,0.0,0.0,1,0,0,0,0


In [55]:
lunch_df = pd.concat([train_df, lunch_menu], axis=1).reset_index(drop=True)
dinner_df = pd.concat([train_df, dinner_menu], axis=1).reset_index(drop=True)

In [56]:
lunch_df

,본사정원수,본사시간외근무명령서승인건수,중식계,석식계,휴일전날,휴일다음날,점심비,저녁비,금,목,...,해조류,과일류,전류,조림류,튀김류,장아찌류,구이류,스프류,덮밥국밥류,떡류
0,2401.0,238,1039.0,331.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2378.0,319,867.0,560.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2365.0,111,1017.0,573.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,2277.0,355,978.0,525.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,2142.0,34,925.0,330.0,1.0,0.0,0.0,0.0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2343.0,704,NaN,NaN,0.0,1.0,0.0,0.0,0,0,...,1,1,0,0,1,0,1,0,0,0
1251,2363.0,636,NaN,NaN,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,0,0,0,0
1252,2329.0,1,NaN,NaN,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1253,2306.0,509,NaN,NaN,0.0,0.0,0.0,0.0,0,1,...,0,1,0,0,0,0,1,0,0,0


## csv 저장

In [57]:
lunch_df.to_csv('lunch_train_test.csv')
dinner_df.to_csv('dinner_train_test.csv')